Importo Librerias

In [4]:
import pandas as pd
import numpy as np 
import ast

Cargo Dataframes de trabajo

In [5]:
df_games = pd.read_parquet(r'D:\projectos\programacion\henrry\bootcamp\proyecto_individuales\1\p1\Proyecto_Individual_N-1_kalchichen_lucas\dataset\ETL_dataset\steam_games_clean.parquet')
df_items = pd.read_parquet(r'D:\projectos\programacion\henrry\bootcamp\proyecto_individuales\1\p1\Proyecto_Individual_N-1_kalchichen_lucas\dataset\ETL_dataset\user_items_cleaned.parquet')
df_reviews = pd.read_parquet(r'D:\projectos\programacion\henrry\bootcamp\proyecto_individuales\1\p1\Proyecto_Individual_N-1_kalchichen_lucas\dataset\ETL_dataset\users_reviews_cleaned.parquet')

Funcion 1

+ def **developer( *`desarrollador` : str* )**:
    `Cantidad` de items y `porcentaje` de contenido Free por año según empresa desarrolladora. 
Ejemplo de retorno:

| Año  | Cantidad de Items | Contenido Free  |
|------|-------------------|------------------|
| 2023 | 50                | 27%              |
| 2022 | 45                | 25%              |
| xxxx | xx                | xx%              |

    

In [61]:
def developer(dev: str):
    df_dev = df_games[df_games['developer'] == dev]
    
    # Calculate the number of items per year
    items_per_year = df_dev.groupby('year').size()
    
    # Calculate the percentage of free content per year
    free_content_per_year = df_dev[df_dev['price'] == 0.0].groupby('year').size() / items_per_year.replace(0, np.nan) * 100
    free_content_per_year = free_content_per_year.fillna(0)  # Replace NaN values with 0
    
    data = []
    for year in items_per_year.index:
        data.append([year, items_per_year[year], free_content_per_year.get(year, 0)])  # Retrieve the value or default to 0
        
    data_return = pd.DataFrame(data, columns=['year', 'items', 'free_content'])
    return data_return


In [62]:
developer('Ubisoft Montreal')


,year,items,free_content
0,2005,2,0.0
1,2006,2,0.0
2,2008,4,0.0
3,2010,4,0.0
4,2011,2,0.0
5,2012,5,0.0
6,2013,13,0.0
7,2014,4,0.0
8,2015,2,50.0
9,2016,24,0.0


Funcion 2

+ def **userdata( *`User_id` : str* )**:
    Debe devolver `cantidad` de dinero gastado por el usuario, el `porcentaje` de recomendación en base a reviews.recommend y `cantidad de items`.

    Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [56]:
def userdata(user_id: str):

    df_games_price = df_games.copy()
    df_user_games = df_items.copy()
    df_user_reviews = df_reviews.copy()
    
    # Filtrar los juegos que posee el usuario en el DataFrame df_games_price
    df_user_games_price = df_games_price[df_games_price['item_id'].isin(df_user_games[df_user_games['user_id'] == user_id]['item_id'])]
    
    # Filtrar las reviews hechas por el usuario en el DataFrame df_reviews
    df_user_reviews = df_reviews[df_reviews['user_id'] == user_id]
    
    # Calcular el dinero total gastado por el usuario
    total_money_spent = df_user_games_price['price'].sum()
    
    # Calcular el porcentaje de recomendación basado en reviews.recommend
    percentage_recommendation = df_user_reviews['recommend'].mean() * 100 if not df_user_reviews.empty else 0
    
    # Obtener el número de items
    num_items = len(df_user_games_price)
    
    return {"Usuario": user_id, "Dinero gastado": total_money_spent, "% de recomendación": f"{percentage_recommendation}%", "Cantidad de items": num_items}

In [57]:
userdata('REBAS_AS_F-T')

{'Usuario': 'REBAS_AS_F-T',
 'Dinero gastado': 14214.63,
 '% de recomendación': '100.0%',
 'Cantidad de items': 1140}

Funcion 3

+ def **UserForGenre( *`genero` : str* )**:
    Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf,
			     "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [44]:
def UserForGenre(genero: str):
    df_genre = df_games.copy()
    df_user = df_items.copy()
    
    # Convertir arrays de NumPy en listas estándar y manejar listas nulas
    df_genre['genres'] = df_genre['genres'].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x if (isinstance(x, list) and x is not None) else [])
    
    # Filtrar el DataFrame para dejar solo los juegos que contengan el género especificado
    df_genre = df_genre[df_genre['genres'].apply(lambda x: genero in x if isinstance(x, list) else False)]

    # Filtrar los usuarios que poseen los juegos del género específico
    df_user_aggregated = df_user[df_user['item_id'].isin(df_genre['item_id'])]
    
    # Merge para concatenar el año de df_genre a df_user_aggregated basado en el item_id
    df_user_aggregated = df_user_aggregated.merge(df_genre[['item_id', 'year']], on='item_id', how='left')
    
    # Calcular la suma de las horas jugadas por cada usuario a los juegos del género específico
    user_hours_per_game = df_user_aggregated.groupby('user_id')['hours_game'].sum()
    
    # Obtener al usuario con más horas jugadas
    user_most_hours_user_id = user_hours_per_game.idxmax()
    
    # Filtrar las horas jugadas por el usuario con más horas jugadas
    user_most_hours_df = df_user_aggregated[df_user_aggregated['user_id'] == user_most_hours_user_id]
    
    # Calcular la cantidad de horas jugadas por año del usuario con más horas jugadas considerando el año de publicación del juego
    hours_per_year = user_most_hours_df.groupby('year')['hours_game'].sum().reset_index()
    
    # Formatear el resultado en el formato especificado
    result = {
        "Usuario con más horas jugadas para " + genero: user_most_hours_user_id,
        "Horas jugadas": [{"Año": int(row['year']), "Horas": int(row['hours_game'])} for index, row in hours_per_year.iterrows()]
    }
    
    return result

    
# Suponiendo que tienes un DataFrame df_games y quieres filtrar por el género 'Action'
df_juegos_genero_action = UserForGenre('Action')

In [45]:
UserForGenre('Indie')

{'Usuario con más horas jugadas para Indie': 'REBAS_AS_F-T',
 'Horas jugadas': [{'Año': 2001, 'Horas': 0},
  {'Año': 2003, 'Horas': 31},
  {'Año': 2005, 'Horas': 36},
  {'Año': 2006, 'Horas': 27},
  {'Año': 2007, 'Horas': 17},
  {'Año': 2008, 'Horas': 22},
  {'Año': 2009, 'Horas': 483},
  {'Año': 2010, 'Horas': 361},
  {'Año': 2011, 'Horas': 1810},
  {'Año': 2012, 'Horas': 2619},
  {'Año': 2013, 'Horas': 3361},
  {'Año': 2014, 'Horas': 6021},
  {'Año': 2015, 'Horas': 13057},
  {'Año': 2016, 'Horas': 14530},
  {'Año': 2017, 'Horas': 564}]}

In [ ]:

# Definir una función que filtra el DataFrame por género
def UserForGenre(genero:str):

    df_genre = df_games.copy()
   # Convertir las cadenas de la columna 'genres' en listas reales
    df_genre['genres'] = df_genre['genres'].apply(ast.literal_eval)

# Ahora puedes aplicar join a la columna 'genres'
    df_genre['genres_str'] = df_genre['genres'].apply(lambda x: ', '.join(x))

    return df_genre

# Ejemplo de uso de la función
# Suponiendo que tienes un DataFrame df_games y quieres filtrar por el género 'Action'
df_juegos_genero_action = UserForGenre('Action')

Funcion 4

+ def **best_developer_year( *`año` : int* )**:
   Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
  
    Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [67]:
def best_developer_year(año: int):
    
    df_reviewed_games = df_games.copy()
    df_rv = df_reviews.copy()
    df_reviewed_games = df_reviewed_games[df_reviewed_games['year'] == año]  # Filtrar por el año dado
    df_reviewed_games = df_reviewed_games.merge(df_rv, on='item_id')  # Combinar con el DataFrame de reviews
    
    # Filtrar por reviews positivas (recommend = True y sentiment_analysis = 2)
    positive_reviews = df_reviewed_games[(df_reviewed_games['recommend'] == True) & (df_reviewed_games['sentiment_analysis'] == 2)]
    
    # Contar los juegos más recomendados por usuarios por desarrollador
    top_developers = positive_reviews.groupby('developer').size().nlargest(3)
    
    # Crear el formato de retorno
    resultado = [{"Puesto 1" : top_developers.index[0]}, {"Puesto 2" : top_developers.index[1]}, {"Puesto 3" : top_developers.index[2]}]
    
    return resultado

In [70]:
best_developer_year(2001)

[{'Puesto 1': 'Firaxis Games'},
 {'Puesto 2': 'Gearbox Software'},
 {'Puesto 3': 'Ensemble Studios'}]

Funcion 5

+ def **developer_reviews_analysis( *`desarrolladora` : str* )**:
    Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total 
    de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo. 

    Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [6]:
def developer_reviews_analysis(desarrolladora: str):
    df_dev = df_games.copy()
    df_dev = df_games[df_games['developer'] == desarrolladora]
    df_dev_reviews = df_reviews[df_reviews['item_id'].isin(df_dev['item_id'])]
    
    # Filtrar las reviews con análisis de sentimiento positivo y negativo
    positive_reviews = df_dev_reviews[df_dev_reviews['sentiment_analysis'] == 2]
    negative_reviews = df_dev_reviews[df_dev_reviews['sentiment_analysis'] == 1]
    
    # Contar el total de reviews positivas y negativas por desarrollador
    positive_count = len(positive_reviews)
    negative_count = len(negative_reviews)
    
    return {desarrolladora: ['Negative = ' + str(negative_count), 'Positive = ' + str(positive_count)]}

In [8]:
developer_reviews_analysis('Gearbox Software')

{'Gearbox Software': ['Negative = 8', 'Positive = 54']}